<a href="https://colab.research.google.com/github/utkarsh-crypto/Gender_classification_VGG-16/blob/main/gender_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

drive_path = "/content/drive/MyDrive/CNN/UTKFace.zip"
local_path = '/content'
!cp '{drive_path}' .
os.chdir(local_path)
!unzip -q 'UTKFace.zip'

In [ ]:
#Give dataset path
train_path = '/content/UTKFace/train'
test_path = '/content/UTKFace/test'

In [ ]:
from PIL import Image 
import os 
from IPython.display import display
from IPython.display import Image as _Imgdis
# creating a object  

  
folder = train_path+'/male'


onlymalefiles = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
print("Working with {0} images".format(len(onlymalefiles)))
print("Image examples: ")


for i in range(10):
    print(onlymalefiles[i])
    display(_Imgdis(filename=folder + "/" + onlymalefiles[i], width=240, height=240))

In [ ]:
num_classes=2


In [ ]:
#model initialization
vgg=VGG16(include_top=False, pooling='avg', weights='imagenet',input_shape=(178, 218, 3))

# Freeze the layers except the last 5
for layer in vgg.layers[:-5]:
 layer.trainable = False
# Check the trainable status of the individual layers
for layer in vgg.layers:
 print(layer, layer.trainable)

vgg.summary()




<keras.engine.input_layer.InputLayer object at 0x7f44d43750d0> False
<keras.layers.convolutional.Conv2D object at 0x7f44d0323e50> False
<keras.layers.convolutional.Conv2D object at 0x7f445e097e50> False
<keras.layers.pooling.MaxPooling2D object at 0x7f445e00fd90> False
<keras.layers.convolutional.Conv2D object at 0x7f445e070510> False
<keras.layers.convolutional.Conv2D object at 0x7f445e05c510> False
<keras.layers.pooling.MaxPooling2D object at 0x7f44d031b910> False
<keras.layers.convolutional.Conv2D object at 0x7f445e065f10> False
<keras.layers.convolutional.Conv2D object at 0x7f445e00e2d0> False
<keras.layers.convolutional.Conv2D object at 0x7f445e02e5d0> False
<keras.layers.pooling.MaxPooling2D object at 0x7f445e027c10> False
<keras.layers.convolutional.Conv2D object at 0x7f445e019890> False
<keras.layers.convolutional.Conv2D object at 0x7f445dfff490> False
<keras.layers.convolutional.Conv2D object at 0x7f445e023350> False
<keras.layers.pooling.MaxPooling2D object at 0x7f445dfff990>

In [28]:
from keras import models
from keras import layers

model = models.Sequential()

# Add new layers for fine-tuning
model.add(vgg)
model.add(Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dense(num_classes, activation='softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense_4 (Dense)             (None, 128)               65664     
                                                                 
 batch_normalization_2 (Batc  (None, 128)              512       
 hNormalization)                                                 
                                                                 
 dense_5 (Dense)             (None, 2)                 258       
                                                                 
Total params: 14,781,122
Trainable params: 7,145,602
Non-trainable params: 7,635,520
___________________________________

In [37]:
#Model Optimizer&Loss set
from keras import optimizers

model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics=['accuracy'])



In [29]:
# use early stopping to optimally terminate training through callbacks
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
es=EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)

# save best model automatically
import h5py
mc= ModelCheckpoint('/content/optimal.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
cb_list=[es,mc]

In [18]:
#Pre-Processing
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)


train_generator = data_generator.flow_from_directory(
        '/content/UTKFace/train',
        target_size=(178, 218),
        batch_size=12,
        class_mode='categorical')


validation_generator = data_generator.flow_from_directory(
        '/content/UTKFace/val',
        target_size=(178, 218),
        batch_size=12,
        class_mode='categorical')





Found 17287 images belonging to 2 classes.
Found 4616 images belonging to 2 classes.


In [27]:

model.fit(
        train_generator,
        epochs=20,
        validation_data=validation_generator,
        callbacks=cb_list)

Epoch 1/20
1441/1441 [==============================] - ETA: 0s - loss: 0.1130 - accuracy: 0.9622
Epoch 00001: val_loss improved from inf to 0.24941, saving model to /content/optimal.h5
1441/1441 [==============================] - 218s 152ms/step - loss: 0.1130 - accuracy: 0.9622 - val_loss: 0.2494 - val_accuracy: 0.9066
Epoch 2/20
1441/1441 [==============================] - ETA: 0s - loss: 0.1068 - accuracy: 0.9647
Epoch 00002: val_loss did not improve from 0.24941
1441/1441 [==============================] - 218s 151ms/step - loss: 0.1068 - accuracy: 0.9647 - val_loss: 4.7146 - val_accuracy: 0.8863
Epoch 3/20
1441/1441 [==============================] - ETA: 0s - loss: 0.0953 - accuracy: 0.9672
Epoch 00003: val_loss did not improve from 0.24941
1441/1441 [==============================] - 219s 152ms/step - loss: 0.0953 - accuracy: 0.9672 - val_loss: 0.3652 - val_accuracy: 0.8878
Epoch 4/20
1441/1441 [==============================] - ETA: 0s - loss: 0.0754 - accuracy: 0.9766
Epoch 0

KeyboardInterrupt: ignored

In [30]:
#load best model
from keras.models import load_model
saved_model = load_model('/content/optimal.h5')

In [35]:
#testing on tuned model
test_generator = data_generator.flow_from_directory(
        '/content/UTKFace/test',
        target_size=(178, 218),
        batch_size=12,
        class_mode='categorical')

# # obtain predicted activation values for the last dense layer
# pred = saved_model.predict(test_generator, verbose=1)
# # determine the maximum activation value for each sample
# predicted_class_indices=np.argmax(pred,axis=1)



Found 1804 images belonging to 2 classes.
151/151 [==============================] - 15s 99ms/step


In [38]:
results = model.evaluate(test_generator,batch_size=12)
results

151/151 [==============================] - 17s 107ms/step - loss: 7628.5205 - accuracy: 0.8603


[7628.5205078125, 0.860310435295105]